##### Load libraries

In [1]:
### Load libraries
import torch
from transformers import  LlamaForCausalLM
import models.Retriever as Retriever
from transformers import  GenerationConfig
from models.Parse import parse_answer, finance_parse_answer, music_parse_answer, sports_parse_answer, open_parse_answer
from models.prompt_api import template_map
import time
from peft import PeftModel
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
torch.cuda.empty_cache()

import bz2
import json
import os
from datetime import datetime
from loguru import logger
from openai import APIConnectionError, OpenAI, RateLimitError
from prompts.templates import IN_CONTEXT_EXAMPLES, INSTRUCTIONS
from tqdm.auto import tqdm

##### load and preprocess data

In [2]:
def load_data_in_batches(dataset_path, batch_size):
    """
    Generator function that reads data from a compressed file and yields batches of data.
    Each batch is a dictionary containing lists of interaction_ids, queries, search results, query times, and answers.
    
    Args:
    dataset_path (str): Path to the dataset file.
    batch_size (int): Number of data items in each batch.
    
    Yields:
    dict: A batch of data.
    """
    def initialize_batch():
        """ Helper function to create an empty batch. """
        return {"interaction_id": [], "query": [], "search_results": [], "query_time": [], "answer": []}

    try:
        with bz2.open(dataset_path, "rt") as file:
            batch = initialize_batch()
            for line in file:
                try:
                    item = json.loads(line)
                    for key in batch:
                        batch[key].append(item[key])
                    
                    if len(batch["query"]) == batch_size:
                        yield batch
                        batch = initialize_batch()
                except json.JSONDecodeError:
                    logger.warn("Warning: Failed to decode a line.")
            # Yield any remaining data as the last batch
            if batch["query"]:
                yield batch
    except FileNotFoundError as e:
        logger.error(f"Error: The file {dataset_path} was not found.")
        raise e
    except IOError as e:
        logger.error(f"Error: An error occurred while reading the file {dataset_path}.")
        raise e

In [3]:
# load and preprocess data
DATASET_PATH = "example_data/dev_data.jsonl.bz2"
batch_size = 1
data_batches = load_data_in_batches(DATASET_PATH, batch_size)


In [4]:
queries, ground_truths, predictions, search_results = [], [], [], []
# batch_size = participant_model.get_batch_size()

for i, batch in enumerate(tqdm(data_batches, desc="Generating predictions")):
	# batch_ground_truths = batch.pop("answer")  # Remove answers from batch and store them
	# batch_predictions = participant_model.batch_generate_answer(batch)
	
	queries.extend(batch["query"])
	ground_truths.extend(batch["answer"])
	search_results.extend(batch["search_results"])
	# predictions.extend(batch_predictions)
	if i >=20:
		break
print(queries, ground_truths)

Generating predictions: 0it [00:00, ?it/s]

['is microsoft office 2019 available in a greater number of languages than microsoft office 2013?', 'what company in the dow jones is the best performer today?', 'name three celebrities who have been involved in successful collaborations with beauty brands.', 'what is the other language between heaven and hell came out in originally?', 'how many times has rory mcilroy won the masters tournament?', 'which companies have the highest level of dividend yield?', 'what is the shortest highway in the us in feet?', 'which company have larger market cap, plya or seatw?', 'who was the first actor to play the role of a comic book villain in a live-action movie?', 'is dreamworks animation owned by time warner or universal pictures?'] ['yes', 'salesforce', 'three celebrities who have been involved in successful collaborations with beauty brands are jennifer aniston (lolavie), kim kardashian (skkn), and selena gomez (rare).', 'en', 'rory mcilroy has never won the masters tournament, despite being on

In [5]:
print(batch.keys())
print(len(search_results[0]))
print(search_results[0][0].keys())
# print(search_results[0][0])

dict_keys(['interaction_id', 'query', 'search_results', 'query_time', 'answer'])
5
dict_keys(['page_name', 'page_url', 'page_snippet', 'page_result', 'page_last_modified'])


In [6]:
index = 1
query = queries[index]
print(query)

search_result_0 = search_results[index]
search_result_cont = []
for page in search_result_0:
	search_result_cont.append(page['page_result'])
	# print(page['page_snippet'])
# text = process_webpage(search_result_0)

what company in the dow jones is the best performer today?


##### Retriever

In [ ]:
from transformers import AutoTokenizer
from langchain_text_splitters import CharacterTextSplitter
from FlagEmbedding import B,FlagReranker

In [ ]:
reranker = FlagReranker(rerank_path, use_fp16=True,device =device2)
        
tokenizer = AutoTokenizer.from_pretrained(token_path)

parent_text_splitter = CharacterTextSplitter(
	chunk_size=parent_chunk_size,
	chunk_overlap=parent_chunk_overlap,
	separator=' '
) 

In [7]:
retriever = Retriever.Retriever2(batch_size=64, device1="cuda:1", device2="cuda:1",
                                hf_path="models/all-Mini-L6-v2", parent_chunk_size=2000, parent_chunk_overlap=400,
                                child_chunk_size=200, child_chunk_overlap=50,
                                )

/home/ahmed.al-ghosaini/Workspace/meta-kdd-cup-24-db-3/models/Retriever.py:44: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.hf_embeddings = HuggingFaceEmbeddings(model_name=hf_path,
No sentence-transformers model found with name models/all-Mini-L6-v2. Creating a new one with MEAN pooling.
/home/ahmed.al-ghosaini/micromamba/envs/default/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of

In [8]:
if retriever.init_retriever(search_result_0, query=query, task3=False):
	search_empty = 0
else:
	search_empty = 1
res = retriever.get_result(query, k=5)
print(res)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


5it [00:00, 34.87it/s]
/home/ahmed.al-ghosaini/Workspace/meta-kdd-cup-24-db-3/models/Retriever.py:249: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  hf_vectorstore = Chroma(


len 4837
len 2920
len 2445
len 3547
get_text 0.14530110359191895
hf_retriever 1.0439143180847168
EnsembleRetriever 1.0439987182617188
len docs 17
["industrial average is one of the oldest stock market indexes. there are 30 dow jones stocks designed to serve as a bellwether for the general u.s. stock market. other major stock indexes include the technology-heavy nasdaq composite and the s&p 500 index — an index of the 500 largest companies in the u.s. best dow jones stocks to watch company name symbol closing price 2023 performance apple ( aapl ) 193.42 +48% intel ( intc ) 41.92 +90% microsoft ( msft ) 372.52 +56% visa ( v ) 254.61 +25% salesforce ( crm ) 251.02 +98% source: ibd data as of dec. 29, 2023 today's dow jones movers inside the dow jones industrial average, walmart ( wmt ) was one of tuesday's biggest gainers, up 1.6%. shares are extended past a cup-with-handle entry at 54.47. on the downside, intel ( intc ) and salesforce ( crm ) were the biggest decliners, both sliding 5%. 

/home/ahmed.al-ghosaini/Workspace/meta-kdd-cup-24-db-3/models/Retriever.py:133: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = self.retriever.get_relevant_documents(query)


In [9]:
# context_str = tokenizer.encode(context_str, max_length=4000, add_special_tokens=False)
context_str = "\n ".join(res)
print(context_str)

industrial average is one of the oldest stock market indexes. there are 30 dow jones stocks designed to serve as a bellwether for the general u.s. stock market. other major stock indexes include the technology-heavy nasdaq composite and the s&p 500 index — an index of the 500 largest companies in the u.s. best dow jones stocks to watch company name symbol closing price 2023 performance apple ( aapl ) 193.42 +48% intel ( intc ) 41.92 +90% microsoft ( msft ) 372.52 +56% visa ( v ) 254.61 +25% salesforce ( crm ) 251.02 +98% source: ibd data as of dec. 29, 2023 today's dow jones movers inside the dow jones industrial average, walmart ( wmt ) was one of tuesday's biggest gainers, up 1.6%. shares are extended past a cup-with-handle entry at 54.47. on the downside, intel ( intc ) and salesforce ( crm ) were the biggest decliners, both sliding 5%. amid the current stock market rally — according to the ibd big picture — investors should focus on stocks that show strong relative strength . these

##### Prompt

In [10]:
system_message = {
    "role": "system",
    "content": (
        "You are an AI assistant specializing in providing concise and accurate answers "
        "to user queries by utilizing the provided context. If the answer is not within "
        "the given context, respond with 'I don't know'. Limit your responses to three sentences."
		"make the answer concise and accurate."
		"make sure the answer is within the context."
		"make the answer as short as possible."
    )
}

user_message = {
    "role": "user",
    "content": (
        "Question: {question}\n"
        "Context: {context}\n"
    ).format(
        question=query,
        context=context_str
    )
}
messages = [
	system_message,
	user_message,
]

##### Model

In [11]:
model_name = "models/Llama-3-8B-instruct"

num_gpus = torch.cuda.device_count()
if num_gpus <= 2:
	model = LlamaForCausalLM.from_pretrained(model_name, device_map="balanced",
												max_memory={0: "44000MiB", 1: 0, "cpu": 0})
	used1 = "cuda:1"
	used2 = "cuda:1"
	used = 'cuda:1'
else:
	model = LlamaForCausalLM.from_pretrained(model_name, device_map="balanced", )
	used1 = "cuda:1"
	used2 = "cuda:2"
	used = "cuda:1"

# model = PeftModel.from_pretrained(model, "models/pretrain_models/llama3-52-peft/checkpoint-480", adapter_name="480").eval()
# model.load_adapter("models/pretrain_models/tran_619_apioutput/checkpoint-310", adapter_name="api_answer")
# model.load_adapter("models/pretrain_models/train_618api_up/checkpoint-580", adapter_name="generate_api")

# model.load_adapter("models/pretrain_models/llama3-52-peft/checkpoint-500", adapter_name="old_generate_api")
# model.load_adapter("models/pretrain_models/llama3-52-peft/checkpoint-580", adapter_name="old_api_answer")
# model.set_adapter("480")

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


##### test

In [12]:
print(query)
# prepare input
input_ids = tokenizer.apply_chat_template(
                messages,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(model.device)

# generate output
outputs =  model.generate(
			input_ids=input_ids,
			# generation_config=generation_config,
			# eos_token_id=terminators,
			return_dict_in_generate=True,
			output_scores=False)
output = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True).lower().split("assistant")[
                -1].strip()
print(output)
# with model.disable_adapter():
# 	model.generate(
# 			input_ids=input_ids,
# 			# generation_config=generation_config,
# 			# eos_token_id=terminators,
# 			return_dict_in_generate=True,
# 			output_scores=False)
# output = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True).lower().split("assistant")[
#                 -1].strip()
# print(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


what company in the dow jones is the best performer today?
the best dow jones stock performer today is intel (intc) with a 5% decline.


In [13]:
print(ground_truths[index])

salesforce


In [ ]:
'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are a helpful, respectful and honest question answering system. I will be providing you with questions, as well as some additional context that may be helpful in answering those questions. You will be provided with additional information between the "<DOCS>" tags. Keep your answers brief, ideally less than 20 words, but a strict limit of 30 words.\n\nIf the provided information is insufficient for answering the question, simply output "Insuffient information" and only output that. If the question asserts a false premise, like "When did Eisenhower become Prime Minister?", simply output "Invalid question". Thus, if you are inclined to say something like "X never has" or "X never was" or "X never did", your output should be "Invalid question".\n\nFinal reminder, with award shows, years can be tricky. Often awards are handed out the year after the project is made. Thus for the Oscars in 2015, the awards are being given out to movies made in 2014.<|eot_id|><|start_header_id|>user<|end_header_id|>\n# Example 1\nQUESTION: who was the first nba player to get a recorded triple double in basketball?\n<DOCS>\n<DOC>\nAndy Phillip got a triple-double versus the Fort Wayne Pistons on December 14, 1950.\n</DOC>\n</DOCS>\nANSWER: andy phillip\n\n# Example 2\nQUESTION: what year did ronald reagan become kommissar in the soviet union?\n<DOCS>\n<DOC>\nRonald Wilson Reagan was an American politician and actor who served as the 40th president of the United States from 1981 to 1989.\n</DOC>\n<DOC>\nReagan\'s policies also helped contribute to the end of the Cold War and the end of Soviet communism.\n</DOC>\n</DOCS>\nANSWER: invalid question\n\n# Example 3\nQUESTION: what are the total number of farms in nebraska?\n<DOCS>\n<DOC>\nThere are many farms in Nebraska, some growing corn, while other growing sugar beats.\n</DOC>\n<DOC>\nNebraska\'s largest industries are the argiculture and cattle production industries.\n</DOC>\n</DOCS>\nANSWER: i don\'t know\n\n<DOCS>\n<DOC>\nDow 30 Stocks List Today • Dogs of the Dow\n## Which Dow stock pays the highest dividend?\n\nOf the 30 Dow Jones Industrial Companies, IBM pays the highest dividend with\nan annual dividend yield of over 5%.\n</DOC>\n<DOC>\nBest Dow Jones Stocks To Watch In 2024\nThe Dow Jones Industrial Average ended February at new highs, as the ongoing\nstock market rally continues. The best Dow Jones stocks to watch in 2024 are\nApple (AAPL), Merck (MRK), Microsoft (MSFT) and Visa (V).\n</DOC>\n<DOC>\nBest Dow Jones Stocks To Watch In 2024\n\n* Research\n\n#  Today\'s Dow Jones Stocks To Watch: Apple Stock Dives On Slowing iPhone\nSales\n\nLicensing\n\n  * SCOTT LEHTONEN\n  * 02:20 PM ET 03/05/2024\n</DOC>\n<DOC>\nBest Dow Jones Stocks To Watch In 2024\n## IBM Stock\nIBM shares are approaching a flat base\'s 196.90 buy point.\nThe stock dipped 0.2% Tuesday.\n## Merck Stock\nDrug giant Merck is breaking out past a 119.65 cup-base buy point.\nMerck shares fell 1% Tuesday, back in the 5% buy area above the entry and\npulling back to the 50-day line.\n</DOC>\n<DOC>\nBest Dow Jones Stocks To Watch In 2024\nThe worst three Dow Jones stocks in 2023 were Walgreens Boots Alliance (WBA),\nChevron (CVX) and Johnson & Johnson (JNJ), with respective declines of 30, 16%\nand 11%.\n</DOC>\n<DOC>\nBest Dow Jones Stocks To Watch In 2024\nOn the downside, Intel (INTC) and Salesforce (CRM) were the biggest decliners,\nboth sliding 5%.\nAmid the current stock market rally — according to the IBD Big Picture —\ninvestors should focus on stocks that show strong relative strength. These\ncould again become 2024 market leaders if the Dow Jones Industrial Average is\nable to extend its recent gains.\n</DOC>\n<DOC>\nBest Dow Jones Stocks To Watch In 2024\nThe best Dow Jones stocks to buy and watch in 2024 include Apple, IBM, Merck,\nMicrosoft and Visa.\nStock Market ETF Strategy And How To Invest\n## Dow Jones Leader: Apple Stock\nAmong Dow Jones stocks in the Magnificent Seven, Apple stock lost 2.9% Tuesday\non a report that showed slowing iPhone sales in China. The company sold 24%\nfewer iPhones in China in the first six weeks of 2024 than a year earlier,\naccording to research firm Counterpoint Research.\n</DOC>\n<DOC>\nBest Dow Jones Stocks To Watch In 2024\n###### Source: IBD Data As Of Dec. 29, 2023\n## Today\'s Dow Jones Movers\nInside the Dow Jones Industrial Average, Walmart (WMT) was one of Tuesday\'s\nbiggest gainers, up 1.6%. Shares are extended past a cup-with-handle entry at\n54.47.\n</DOC>\n<DOC>\nBest Dow Jones Stocks To Watch In 2024\nX\nThere are clear winners — and losers — at the start of February. The top three\nperforming blue chip stocks in 2023 were Salesforce (CRM), Intel (INTC) and\nMicrosoft, posting rallies of 98%, 90% and 56%, respectively.\n</DOC>\n<DOC>\nTop 30 Companies of Dow Jones Index by Weight in 2023\n## Best Dow Jones ETF\nInvesting in an ETF (Exchange-Traded Fund) is another way to invest in the Dow\nJones index. The best Dow Jones ETF in the market today is SPDR Dow Jones\nIndustrial Average ETF (DIA). It is a non-leveraged ETF that copies the\nperformance of the Dow and its stock allocation.\n</DOC>\n</DOCS>\nQUESTION (asked at 03/05/2024, 23:18:31 PT): what company in the dow jones is the best performer today?<|eot_id|><|start_header_id|>assistant<|end_header_id|>ANSWERABLE:'

In [18]:
import os
from openai import OpenAI
os.environ["INTERWEB_APIKEY"] = "D40tLQc4plxanXT91P3zEJ2Dk1mjNUJOhjxT7uCKkZPgFq1NO1Ew8ZLI47KICpku"
os.environ['INTERWEB_HOST'] = "https://interweb.l3s.uni-hannover.de"
# os.environ['INTERWEB_HOST'] = "http://gpunode04.kbs:11434/v1/"
# os.environ['INTERWEB_APIKEY'] = "ollama"

client = OpenAI(
    base_url=os.getenv("INTERWEB_HOST", "http://gpunode04.kbs:11434/v1/"),
    api_key=os.getenv("INTERWEB_APIKEY"),
)

# List available models
# models = client.models.list()
# print(models)

# Create a completion
message_text = [{
    "role": "system",
    "content": "You are an AI assistant that helps people find information."
}]

In [19]:
for model in models:
	print(model.id)

gpt-35-turbo
gpt-35-turbo-16k
gpt-35-turbo-1106
gpt-4
gpt-4-turbo
gpt-4o
gpt-4o-mini
llama3.3:70b
llama3.2:latest
llama3.2:3b
nomic-embed-text:v1.5
bge-m3:567m
snowflake-arctic-embed:335m
llama3.2-vision:11b-instruct-q8_0
llama3.2-vision:11b
snowflake-arctic-embed:110m
snowflake-arctic-embed:33m
all-minilm:33m
gemma2:2b-instruct-fp16
mistral:7b-instruct-fp16
mistral-nemo:12b-instruct-2407-fp16
gemma2:9b-instruct-fp16
phi3.5:3.8b-mini-instruct-fp16
llama3.2:3b-instruct-fp16
llama3.2:1b-instruct-fp16
llama3.1:8b-instruct-fp16
gemma:7b
llama3.1:8b-instruct-q8_0
llama3.1:70b
mistral-nemo:12b
gemma2:9b
phi3:14b
gemma2:27b
llama3.1:8b
qwen2.5-coder:7b
gemma2:2b
reflection:70b
qwen2.5:7b
qwen2.5:72b
llava:7b
llava:34b
bakllava:7b
llava-llama3:8b
codellama:7b
codegemma:7b
mixtral:8x7b


In [26]:
model_name = "llama3.1:8b-instruct-q8_0" #"llama3.3:70b"
message_text = [
	{
		"role": "system",
		"content": "You are a helpful assistant that provides concise and accurate answers to user questions. You can answer questions, provide explanations, and assist with tasks across various topics."
    },
    {
    "role": "user",
    "content": "You are an AI assistant that helps people find information. Tell me the number of days in a month"
}]

# completion = client.chat.completions.create(
#     model=model_name, # gpt-35-turbo
#     messages=message_text,

# )
completion = client.chat.completions.create(
			model=model_name, # gpt-35-turbo
			messages=message_text,
			# temperature=0.7,
			max_tokens=800,
			# top_p=0.95,
			# frequency_penalty=0,
			# presence_penalty=0,
			# stop=None
		)
# print('## user:', message_text[0]['content'])
print('## assistant:', completion.choices[0].message.content)

# print(completion.choices[0].message.content)

## assistant: There are 31 days in January, March, May, July, August, October, and December.
There are 30 days in April, June, September, and November.
February has 28 days (29 days in leap years).


In [29]:
model_name = "llama3.1:8b-instruct-fp16"
message_text = [
	{
		"role": "system",
		"content": '''You are a helpful, respectful and honest question answering system. I will be providing you with questions, as well as some additional context that may be helpful in answering those questions. You will be provided with additional information between the "<DOCS>" tags. Keep your answers brief, ideally less than 20 words, but a strict limit of 30 words.

        If the provided information is insufficient for answering the question, simply output "Insuffient information" and only output that. If the question asserts a false premise, like "When did Eisenhower become Prime Minister?", simply output "Invalid question". Thus, if you are inclined to say something like "X never has" or "X never was" or "X never did", your output should be "Invalid question". If you don't know the answer or the question is unanswerable, simply output "I don't know".'''
    },
    {
    "role": "user",
    "content": ''' 
    # Example 1
    QUESTION: who was the first nba player to get a recorded triple double in basketball?
    <DOCS>
    <DOC>
    Andy Phillip got a triple-double versus the Fort Wayne Pistons on December 14, 1950.
    </DOC>
    </DOCS>
    ANSWER: andy phillip

    # Example 2
    QUESTION: what year did ronald reagan become kommissar in the soviet union?
    <DOCS>
    <DOC>
    Ronald Wilson Reagan was an American politician and actor who served as the 40th president of the United States from 1981 to 1989.
    </DOC>
    <DOC>
    Reagan's policies also helped contribute to the end of the Cold War and the end of Soviet communism.
    </DOC>
    </DOCS>
    ANSWER: invalid question

    # Example 3
    QUESTION: what are the total number of farms in nebraska?
    <DOCS>
    <DOC>
    There are many farms in Nebraska, some growing corn, while other growing sugar beats.
    </DOC>
    <DOC>
    Nebraska's largest industries are the argiculture and cattle production industries.
    </DOC>
    </DOCS>
    ANSWER: i don't know

    <DOCS>
    <DOC>
    Steve Nash - Wikipedia
    #### 2013–14 season
    Nash with the Lakers in 2013
    During the 2013–14 season, Nash continued to suffer nerve problems stemming
    from his leg injury the prior season. In November 2013, he was sidelined for
    an estimated minimum of two weeks due to nerve root irritation.[70] Nash
    returned on 4 February 2014, shooting 3-for-6 for 7 points. On 7 February
    2014, his 40th birthday, he scored a season-high 19 points in a victory over
    the Philadelphia 76ers.[71] On 13 March, Nash was ruled out for the remainder
    of the season by D'Antoni. He was suffering from a recurrence of nerve
    problems stemming from an earlier collision to his left leg with Chicago's
    Kirk Hinrich, and the Lakers wanted to give Kendall Marshall and Jordan Farmar
    more playing time. However, Nash returned on 21 March after a groin injury to
    Farmar that was expected to sideline him a minimum of two weeks.[72][73]
    </DOC>
    <DOC>
    Steve Nash - Wikipedia
    ## Player profile
    Nash is known for his dribbling, passing, and shooting abilities.
    Nash was praised for his playmaking, ball-handling skills and shooting. He led
    the league in assists for five years, averaging 11.5 assists per game in
    2004–05, 10.5 in 2005–06, 11.6 in 2006–07, 11.0 in 2009–10 and 11.4 in
    2010–11[16] and won the NBA Skills Challenge in 2005 and 2010.[12] As of the
    end of 2012–13 season, he had a 90.4% career free throw shooting average
    (formerly the best in NBA history, as of 2013[update] second to Stephen
    Curry)[107][108] and a 42.8% career three-point shooting average (eighth-best
    in league history),[109] and ranked as one of the top 10 players in league
    history in total assists, assists per game, and three-point field goals
    made.[110][111][112] He is ranked second (starting from 1986–87) in regular
    season point–assist double doubles.[113] In the 2005–06 season, Nash became
    the fourth player in NBA history to shoot 50% or better from the field, 40%
    from three-point range (43.9), and 90% from the line, joining Larry Bird,
    Reggie Miller, and Mark Price in the 50–40–90 club.[33][114] Nash would repeat
    this feat three more times in the 2007–08, 2008–09 and 2009–10 campaigns.[16]
    Nash (four times), Larry Bird and Kevin Durant (two times each) are the only
    players to have accomplished this feat more than once.[115]
    </DOC>
    <DOC>
    Steve Nash - Wikipedia
    #### 2011–12 season
    In February 2012, Nash was named to his eighth All-Star Game. At the time, he
    was leading the NBA in assists per game.[49] On 21 April 2012, Nash passed
    Oscar Robertson for career assists versus the Denver Nuggets. Despite his
    stellar play the Suns missed the 2012 playoffs for the second consecutive
    time. He finished the season averaging 12.5 points and 10.7 assists per game
    on 53.2% shooting from the field (tying his career high). Near the end of the
    2011–12 season, Nash was named the winner of the PBWA's Magic Johnson Award.
    </DOC>
    <DOC>
    Steve Nash - Wikipedia
    Nash originally attended Mount Douglas Secondary School in Saanich, British
    Columbia, but after his grades began to drop, his parents decided to enroll
    him at St. Michaels University School, a private school in Victoria.[11]
    There, he starred in basketball, soccer, and rugby union. While playing
    basketball during his senior season, Nash averaged 21.3 points, 11.2 assists,
    and 9.1 rebounds per game.[12] In the 1991–92 season, he led his team in his
    final year to the British Columbia AAA provincial championship title, and was
    named the province's Player of the Year.[13]
    </DOC>
    <DOC>
    Steve Nash - Wikipedia
    In his rookie season, Nash only managed 10.5 minutes a game,[16] but in his
    second season, his playing time increased significantly and he was even ranked
    13th in the league for three-point field-goal percentage.[8] Nevertheless, the
    Canadian's tenure with the Suns did not last. Nelson had just taken a job as
    assistant general manager of the Dallas Mavericks under his father, Don
    Nelson, and convinced him to acquire the under-utilised Nash.[17] Following
    the 1998 NBA draft, Nash was traded from the Suns to the Mavericks in exchange
    for Martin Müürsepp, Bubba Wells, the draft rights to Pat Garrity and a first-
    round draft pick (who later turned out to be future Phoenix teammate Shawn
    Marion).[18]
    </DOC>
    <DOC>
    Steve Nash - Wikipedia
    ### Dallas Mavericks (1998–2004)
    It was in Dallas that Nash established himself as a formidable point guard,
    beginning a decade as one of the game's top players. During his first year as
    a Maverick (the lockout-shortened 1998–99 season) he started in all 40 games
    he played in, and averaged 7.9 points, 2.9 rebounds and 5.5 assists per
    game.[16] The 19–31 Mavericks failed to make the 1999 playoffs, but in the
    1999–2000 season, the team's prospects improved considerably. Nash missed 25
    mid-season games due to an ankle injury, but came back to record six double-
    doubles in the last month of play.[12] He finished the season with averages of
    8.6 points and 4.9 assists per game.[16] More importantly for the team,
    second-year teammate and friend Dirk Nowitzki was blossoming quickly into a
    top player, veteran Michael Finley was having an All-Star-calibre year, and
    the team's new owner, billionaire Mark Cuban, was bringing new energy and
    excitement to the franchise. Nash now had a supportive environment in which he
    could thrive.
    </DOC>
    <DOC>
    Steve Nash - Wikipedia
    In the 2000–01 season, Nash averaged 15.6 points and 7.3 assists per game in a
    breakout season.[16] With Nash directing the team's offense, Nowitzki and
    Finley playing at their best, and the acquisition of All-Star Juwan Howard
    complementing the high-scoring trio, the Mavericks earned a playoff berth for
    the first time in more than a decade. Dallas lost in the Western Conference
    Semifinals four games to one to the San Antonio Spurs, but it marked the
    beginning of a memorable run for Nash and the Mavericks.[19] In the 2001–02
    season, Nash posted career-highs of 17.9 points and 7.7 assists per game[16]
    and earned a spot in the NBA All-Star Game and on the All-NBA Third
    Team.[20][21] He was now an All-Star, increasingly appearing in television
    commercials and, with Finley and Nowitzki, a part of the Dallas Mavericks "Big
    Three".[22] Dallas earned another trip to the playoffs but lost again in the
    Semifinals to the Sacramento Kings four games to one.[23]
    </DOC>
    <DOC>
    Steve Nash - Wikipedia
    Nash closely replicated his previous season's performance in the 2002–03
    season, averaging 17.7 points and 7.3 assists per game,[16] again earning All-
    Star and All-NBA Third Team honours.[20][24] Nowitzki and Nash led the
    Mavericks from a 14-game winning streak to open the season all the way to the
    Western Conference finals, where they lost to the eventual NBA champions, the
    San Antonio Spurs four games to two.[25] It was only the second Conference
    finals appearance in the franchise's history. The 2003–04 season saw an
    offensively boosted Mavericks roster (with the acquisitions of Antoine Walker
    and Antawn Jamison) but a dip in Nash's scoring contributions. As a result, he
    was not selected for the All-Star and All-NBA team rosters even though he
    achieved new career highs in assists per game (8.8) and free throw accuracy
    (91.6%).[16] In the 2004 playoffs, the fifth-seeded Dallas failed to make
    progress yet again as the Sacramento Kings saw them off four games to one.[26]
    </DOC>
    <DOC>
    Steve Nash - Wikipedia
    #### 2006–07 season
    In the 2006–07 season, Nash had another stellar campaign, averaging 18.6
    points and a career-high 11.6 assists per game while becoming the first person
    since Magic Johnson in 1990–91 to average 18 points and 11 assists per game
    during the regular season.[38] Nash received the most votes for first-team
    All-NBA and was joined by teammate Stoudemire; the two were the first
    teammates to make the first team since Kobe Bryant and Shaquille O'Neal in
    2003–04.[38] Nash received 129 first-place votes and 645 total points from the
    panel of 129 media members.[38] He narrowly missed being MVP a third
    consecutive time, coming in second with 44 first place votes to 83 for Dirk
    Nowitzki.[39] In the 2007 playoffs, the Suns eliminated the Lakers in five
    games before losing 4–2 to the Spurs in the conference semifinals.[40]
    </DOC>
    </DOCS>
    QUESTION (asked at 03/10/2024, 23:19:21 PT): how many 3-point attempts did steve nash average per game in seasons he made the 50-40-90 club?
    '''
}]

completion = client.chat.completions.create(
    model=model_name, # gpt-35-turbo
    messages=message_text,
    # temperature=0.7,
    max_tokens=800,
    # top_p=0.95,
    # frequency_penalty=0,
    # presence_penalty=0,
    stop=None
)

# print('## user:', message_text[0]['content'])
print('## assistant:', completion.choices[0].message.content)

# print(completion.choices[0].message.content)

## assistant: Unfortunately, I am unable to provide the exact answer because I cannot find this specific information.
